In [4]:
from qcg.generators import gen_supremacy, gen_hwea
from qiskit import BasicAer, execute
import time
from scipy.sparse import csr_matrix
import numpy as np
import random

def measure_basis(l):
    H = [[1,1],[1,-1]]/np.sqrt(2)
    sDagger = [[1,0],[0,-1j]]
    Id = [[1,0],[0,1]]
    if len(l)==1:
        if l[0] == 1 or l[0] == 2 or l[0] == 7 or l[0] == 8:
            return Id
        elif l[0] == 3 or l[0] == 4:
            return H
        elif l[0] == 5 or l[0] == 6:
            return np.matmul(H,sDagger)
        else:
            raise Exception('Illegal change basis')
    else:
        return np.kron(measure_basis([l[0]]), measure_basis(l[1:]))

repeat = int(1e2)
dimensions = [3,3,8]
num_qubits = dimensions[0]*dimensions[1]

start = time.time()
for i in range(repeat):
    circ = gen_supremacy(*dimensions)
    backend = BasicAer.get_backend('statevector_simulator')
    job = execute(circ, backend=backend)
    result = job.result()
    outputstate = result.get_statevector(circ)
end = time.time()
print(end-start)

start = time.time()
for i in range(repeat):
    l = [random.randint(1,8) for i in range(num_qubits)]
    change_basis_matrix = measure_basis(l)
    sparse_matrix = csr_matrix(change_basis_matrix)
    modified_instance = csr_matrix.dot(sparse_matrix, outputstate)
end = time.time()
print(end-start)

start = time.time()
for i in range(repeat):
    l = [random.randint(1,8) for i in range(num_qubits)]
    change_basis_matrix = measure_basis(l)
    modified_instance = np.matmul(change_basis_matrix, outputstate)
end = time.time()
print(end-start)

8.219146013259888
0.38608717918395996
0.17391586303710938
